In [1]:
import json
import os.path
import csv
import pandas as pd
import pylab as pl

In [2]:
NUM_USERS = 400
user_data = {}
call_log = {}
contact_list = {}
sms_log = {}
user_status = {}
#initialize call logs, contact lists and sms logs to empty list
for i in range(1, NUM_USERS+1):
    call_log[i] = []
    contact_list[i] = []
    sms_log[i] = []

#### Get the status of each user and save it as a dictionary

In [3]:
user_status['user_id'] = []
user_status['disbursement_date'] = []
user_status['status'] = []
with open("../data/user_logs/user_status.csv", 'rb') as csvfile:
    has_header = csv.Sniffer().has_header(csvfile.read(1024))
    csvfile.seek(0)  # Rewind
    reader = csv.reader(csvfile)
    if has_header:
        next(reader)  # Skip header row
    for row in reader:
        user_status['user_id'].append(row[0])
        user_status['disbursement_date'].append(row[1])
        user_status['status'].append(row[2])

#### Get the call log, contact list, and sms log for each user and store them in their corresponding dictionaries.  While we're at it, get the number of devices

In [4]:
user_data["num_devices"] = []
for user_id in range(1, NUM_USERS+1):
    path = './../data/user_logs/user-' + str(user_id) + '/'
    devices = [name for name in os.listdir(path)]
    for device in devices:
        call_log_path = path + device + '/collated_call_log.txt'
        contact_list_path = path + device + '/collated_contact_list.txt'
        sms_log_path = path + device + '/collated_sms_log.txt'
        try:
            with open(call_log_path) as call_log_file:
                call_log[user_id].extend(json.load(call_log_file))
        except IOError:
            print "Error loading file"
        try:
            with open(contact_list_path) as contact_list_file:
                contact_list[user_id].extend(json.load(contact_list_file))
        except IOError:
            print "Error loading file"
        try:
            with open(sms_log_path) as sms_log_file:
                sms_log[user_id].extend(json.load(sms_log_file))
        except IOError:
            print "Error loading file"
            
    #print str(user_id) + ":",str(len(devices)), "devices"        
    user_data["num_devices"].append(len(devices))

Error loading file
Error loading file
Error loading file
Error loading file
Error loading file
Error loading file
Error loading file
Error loading file


#### Get some user statistics: number of calls, number of sms messages, number of contacts, number of devices

In [5]:
user_data["num_calls"] = []
user_data["num_contacts"] = []
user_data["num_sms"] = []
user_data["user_id"] = []
for user_id in range(1, NUM_USERS+1):
    user_data["user_id"].append(user_id)
    user_data["num_calls"].append(len(call_log[user_id]))
    user_data["num_contacts"].append(len(contact_list[user_id]))
    user_data["num_sms"].append(len(sms_log[user_id]))

In [6]:
user_data_df = pd.DataFrame(data=user_data)
user_data_df.head()

,num_calls,num_contacts,num_devices,num_sms,user_id
0,182,246,1,133,1
1,1423,88,1,3773,2
2,442,200,1,125,3
3,1775,711,1,10522,4
4,2356,128,1,1654,5


#### Let's see what this dataset looks like

In [10]:
user_data_df[['num_calls','num_contacts','num_sms','num_devices']].describe()

,num_calls,num_contacts,num_sms,num_devices
count,400.000000,400.000000,400.000000,400.000000
mean,1588.225000,573.585000,3336.160000,1.255000
std,1704.651581,1027.860431,6028.870615,0.718516
min,4.000000,14.000000,4.000000,1.000000
25%,500.000000,209.000000,410.250000,1.000000
50%,1000.000000,344.500000,1571.500000,1.000000
75%,2137.250000,594.750000,4443.500000,1.000000
max,11821.000000,13850.000000,86546.000000,7.000000


In [ ]:
user_data_df[['num_calls','num_contacts','num_sms','num_devices']].hist()
pl.show()